In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import pickle
from tqdm import tqdm
import tensorflow as tf

In [2]:
with open('./x.pkl', 'rb') as fp:
    x = pickle.load(fp)
with open('./y.pkl', 'rb') as fp:
    y = pickle.load(fp)
with open('./xpos.pkl', 'rb') as fp:
    xpos = pickle.load(fp)
with open('./ypos.pkl', 'rb') as fp:
    ypos = pickle.load(fp)

In [3]:
pos_index = {}
for xx, yy in zip(xpos, ypos):
    xx = tuple(xx)
    pos_index[xx] = yy

In [4]:
print(len(pos_index))

232857


In [5]:
len(x)

1513223

In [6]:
xbad, ybad = [], []
for xx, yy in tqdm(zip(x, y)):
    xx = tuple(xx)
    if xx not in pos_index:
        xbad.append(xx)
        ybad.append(yy)

1513223it [00:02, 719138.43it/s]


In [7]:
print(len(xbad))

1280366


In [8]:
def cut(model, sent: str):
    if len(sent) <= 0:
        return []
    elif len(sent) > 510:
        sent = sent[:510]
    tokens = ['[CLS]'] + list(sent) + ['[SEP]']
    for lens in (32, 64, 128, 256, 512):
        if len(tokens) <= lens:
            tokens += [''] * (lens - len(tokens))
            break

    inputs = tf.constant([tokens])
    pred = model(inputs)

    words = []
    poses = []
    last_word = ''
    last_pos = ''

    pred_iter = zip(
        sent,
        pred.numpy()[0]
    )
    
    for w, x in pred_iter:
        x = x.decode('utf-8')
        pos = x[1:]
        if x[0] == 'B' or x[0] == 'S':
            if len(last_word):
                words.append(last_word)
                poses.append(last_pos)
                last_word = ''
                last_pos = ''
            last_word += w
            last_pos = pos
        else:
            last_word += w

    if len(last_word):
        words.append(last_word)
        poses.append(last_pos)

    return words, poses

In [9]:
model = tf.saved_model.load(
    'chn_pos_albert')

In [10]:
cut(model, '我爱你')

(['我', '爱', '你'], ['r', 'v', 'r'])

In [11]:
new_xbad = []
new_ybad = []
for xx, yy in tqdm(zip(xbad, ybad), total=len(xbad)):
    xx = xx[:510]
    yy = yy[:510]
    words, pos = cut(model, ''.join(xx))
    cp = []
    for w, p in zip(words, pos):
        for ww in w:
            cp.append((ww, p))
    last_pos = None
    nyy = []
    for xxx, yyy, (_, p) in zip(xx, yy, cp):
        if yyy[0] == 'B':
            last_pos = p
        nyy.append(yyy + last_pos)
    assert len(nyy) == len(yy)
    new_ybad.append(nyy)

  0%|          | 0/1280366 [00:00<?, ?it/s]

  0%|          | 4/1280366 [00:00<55:34:08,  6.40it/s]

100%|██████████| 1280366/1280366 [6:13:33<00:00, 57.13it/s]  


In [12]:
print(len(new_xbad), len(new_ybad))

0 1280366


In [13]:
new_xbad = [xx[:510] for xx in tqdm(xbad)]

100%|██████████| 1280366/1280366 [00:00<00:00, 3303718.66it/s]


In [14]:
print(len(new_xbad), len(new_ybad))

1280366 1280366


In [15]:
for xx, yy in tqdm(zip(new_xbad, new_ybad)):
    assert len(xx) == len(yy)

1280366it [00:00, 2137994.39it/s]


In [16]:
assert len(x) == len(xpos) + len(new_xbad)

In [17]:
xpos += new_xbad

In [18]:
ypos += new_ybad

In [19]:
assert len(x) == len(xpos)

In [24]:
print(len(xpos), len(ypos))

1513223 1513223


In [20]:
with open('sxpos.pkl', 'wb') as fp:
    pickle.dump(xpos, fp)

with open('sypos.pkl', 'wb') as fp:
    pickle.dump(ypos, fp)

In [21]:
!du -sh sxpos.pkl
!du -sh sypos.pkl

534M	sxpos.pkl
506M	sypos.pkl


In [22]:
!du -sh x.pkl
!du -sh y.pkl
!du -sh xpos.pkl
!du -sh ypos.pkl

535M	x.pkl
94M	y.pkl
123M	xpos.pkl
116M	ypos.pkl
